In [1]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#====================100测试数据==============
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools
import copy


start = time.clock()
t0 = time.time()



def splitlines(x):
    y = ' '.join(x.split())
    return y
    


def prune(transactions,Ck,support):
    # CSups candidate and supports
    CSups = {}
    for transaction in transactions:
        for candi in Ck:
            candi = frozenset(candi)
            if candi.issubset(frozenset(transaction)):
              if not frozenset(candi) in CSups:
                  CSups[frozenset(candi)] =1
              else:
                  CSups[frozenset(candi)] +=1

                  
    numRecords = float(len(transactions))
    
    # 满足最小支持度的频繁项集和频率
    # FreSups frequent itemset and support
    FreSups = {}
    CurFre = []
    for candi in CSups:
        support_candi = CSups[candi]/numRecords   
        if support_candi >= support:
            CurFre.append(candi)
            FreSups[candi] = support_candi
     
    return CurFre, FreSups
  
def genCk(Lk, k):
    Ck = [] 
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            L1 = list(set(Lk[i]))[: k-2]
            L2 = list(set(Lk[j]))[: k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                Ck.append(frozenset(Lk[i] | Lk[j]))
    return Ck

def FindFreInPartition(iteration,support):
  import itertools
  frqtemp=[]
  preFreSets=[]
  #support = 0.3
  splitedline=list(iteration)
  transactions=copy.deepcopy(splitedline)
  
  numRecords = len(splitedline)
  freqThre = numRecords * support
  #print ("numRecords：" + str(numRecords))
  #print ("frequency threshold：" + str(freqThre))
  #transactions = list(map(lambda x : x.split(' '),splitedline))
                 
         
  #Create candidate_1
  candidate_1 = list(set(list(itertools.chain.from_iterable(transactions))))#distinct
  candidate_1 = list(map(lambda c: [c] ,candidate_1))
  #print(candidate_1)
  CurFre1, FreSups = prune(transactions,candidate_1,support)
  frqtemp.append(frozenset(CurFre1))
  preFreSets = CurFre1
  
         
  #Create CK LK
  K =200 #迭代次数
  for r in range(2,K):
    #print ('round'+str(r)+':\n')
    #print (preFreSets)
    if len(preFreSets)==0:
      break
    candidates = genCk(preFreSets, r)
    CurFre, FreSups = prune(transactions,candidates,support)
    if len(CurFre)!=0:
      frqtemp.append(CurFre)
    preFreSets = CurFre
                    
  return frqtemp
    

     


support = 0.05


#将输入数据分区，由于后面要频繁使用。因此缓存起来
#display(dbutils.fs.ls("/FileStore/tables"))
splitedl = sc.textFile('/FileStore/tables/num_new_100.txt',8)
#print(input_rdd.collect())


splitedl = splitedl.map(lambda x:splitlines(x))
splitedl.collect()
transationss = splitedl.map(lambda x: x.split(' '))
transationss.collect()
#print('tran:\n',transationss.collect())
#print('spl:\n',splitedl.collect())

numRecords = float(transationss.count())
freqThre = numRecords * support



######
#print('stage:merge')
"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates = transationss.mapPartitions(lambda x: FindFreInPartition(x,support))
#print('globalcan:\n',global_candidates.collect())  
global_candidates = global_candidates.flatMap(lambda its:[frozenset(it) for it in its]).distinct()
#print('globalcan:\n',global_candidates.collect())    

"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
original_data=sc.broadcast(transationss.collect())
    
"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates.map(lambda x:(x,len([transaction for transaction in original_data.value if x.issubset(frozenset(transaction))])/numRecords))\
    .filter(lambda x: x[1]>=support).collectAsMap()
print ('global_frequent_itemsets:\n',global_frequent_itemsets)

  

   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




In [2]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#====================500测试数据==============
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools
import copy


start = time.clock()
t0 = time.time()



def splitlines(x):
    y = ' '.join(x.split())
    return y
    


def prune(transactions,Ck,support):
    # CSups candidate and supports
    CSups = {}
    for transaction in transactions:
        for candi in Ck:
            candi = frozenset(candi)
            if candi.issubset(frozenset(transaction)):
              if not frozenset(candi) in CSups:
                  CSups[frozenset(candi)] =1
              else:
                  CSups[frozenset(candi)] +=1

                  
    numRecords = float(len(transactions))
    
    # 满足最小支持度的频繁项集和频率
    # FreSups frequent itemset and support
    FreSups = {}
    CurFre = []
    for candi in CSups:
        support_candi = CSups[candi]/numRecords   
        if support_candi >= support:
            CurFre.append(candi)
            FreSups[candi] = support_candi
     
    return CurFre, FreSups
  
def genCk(Lk, k):
    Ck = [] 
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            L1 = list(set(Lk[i]))[: k-2]
            L2 = list(set(Lk[j]))[: k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                Ck.append(frozenset(Lk[i] | Lk[j]))
    return Ck

def FindFreInPartition(iteration,support):
  import itertools
  frqtemp=[]
  preFreSets=[]
  #support = 0.3
  splitedline=list(iteration)
  transactions=copy.deepcopy(splitedline)
  
  numRecords = len(splitedline)
  freqThre = numRecords * support
  #print ("numRecords：" + str(numRecords))
  #print ("frequency threshold：" + str(freqThre))
  #transactions = list(map(lambda x : x.split(' '),splitedline))
                 
         
  #Create candidate_1
  candidate_1 = list(set(list(itertools.chain.from_iterable(transactions))))#distinct
  candidate_1 = list(map(lambda c: [c] ,candidate_1))
  #print(candidate_1)
  CurFre1, FreSups = prune(transactions,candidate_1,support)
  frqtemp.append(frozenset(CurFre1))
  preFreSets = CurFre1
  
         
  #Create CK LK
  K =200 #迭代次数
  for r in range(2,K):
    #print ('round'+str(r)+':\n')
    #print (preFreSets)
    if len(preFreSets)==0:
      break
    candidates = genCk(preFreSets, r)
    CurFre, FreSups = prune(transactions,candidates,support)
    if len(CurFre)!=0:
      frqtemp.append(CurFre)
    preFreSets = CurFre
                    
  return frqtemp
    

     


support = 0.01


#将输入数据分区，由于后面要频繁使用。因此缓存起来
#display(dbutils.fs.ls("/FileStore/tables"))
splitedl = sc.textFile('/FileStore/tables/num_500.txt',8)
#print(input_rdd.collect())


splitedl = splitedl.map(lambda x:splitlines(x))
splitedl.collect()
transationss = splitedl.map(lambda x: x.split(' '))
transationss.collect()
#print('tran:\n',transationss.collect())
#print('spl:\n',splitedl.collect())

numRecords = float(transationss.count())
freqThre = numRecords * support



######
#print('stage:merge')
"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates = transationss.mapPartitions(lambda x: FindFreInPartition(x,support))
#print('globalcan:\n',global_candidates.collect())  
global_candidates = global_candidates.flatMap(lambda its:[frozenset(it) for it in its]).distinct()
#print('globalcan:\n',global_candidates.collect())    

"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
original_data=sc.broadcast(transationss.collect())
    
"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates.map(lambda x:(x,len([transaction for transaction in original_data.value if x.issubset(frozenset(transaction))])/numRecords))\
    .filter(lambda x: x[1]>=support).collectAsMap()
print ('global_frequent_itemsets:\n',global_frequent_itemsets)

  

   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




In [3]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#====================500测试数据==============
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools
import copy


start = time.clock()
t0 = time.time()



def splitlines(x):
    y = ' '.join(x.split())
    return y
    


def prune(transactions,Ck,support):
    # CSups candidate and supports
    CSups = {}
    for transaction in transactions:
        for candi in Ck:
            candi = frozenset(candi)
            if candi.issubset(frozenset(transaction)):
              if not frozenset(candi) in CSups:
                  CSups[frozenset(candi)] =1
              else:
                  CSups[frozenset(candi)] +=1

                  
    numRecords = float(len(transactions))
    
    # 满足最小支持度的频繁项集和频率
    # FreSups frequent itemset and support
    FreSups = {}
    CurFre = []
    for candi in CSups:
        support_candi = CSups[candi]/numRecords   
        if support_candi >= support:
            CurFre.append(candi)
            FreSups[candi] = support_candi
     
    return CurFre, FreSups
  
def genCk(Lk, k):
    Ck = [] 
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            L1 = list(set(Lk[i]))[: k-2]
            L2 = list(set(Lk[j]))[: k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                Ck.append(frozenset(Lk[i] | Lk[j]))
    return Ck

def FindFreInPartition(iteration,support):
  import itertools
  frqtemp=[]
  preFreSets=[]
  #support = 0.3
  splitedline=list(iteration)
  transactions=copy.deepcopy(splitedline)
  
  numRecords = len(splitedline)
  freqThre = numRecords * support
  #print ("numRecords：" + str(numRecords))
  #print ("frequency threshold：" + str(freqThre))
  #transactions = list(map(lambda x : x.split(' '),splitedline))
                 
         
  #Create candidate_1
  candidate_1 = list(set(list(itertools.chain.from_iterable(transactions))))#distinct
  candidate_1 = list(map(lambda c: [c] ,candidate_1))
  #print(candidate_1)
  CurFre1, FreSups = prune(transactions,candidate_1,support)
  frqtemp.append(frozenset(CurFre1))
  preFreSets = CurFre1
  
         
  #Create CK LK
  K =200 #迭代次数
  for r in range(2,K):
    #print ('round'+str(r)+':\n')
    #print (preFreSets)
    if len(preFreSets)==0:
      break
    candidates = genCk(preFreSets, r)
    CurFre, FreSups = prune(transactions,candidates,support)
    if len(CurFre)!=0:
      frqtemp.append(CurFre)
    preFreSets = CurFre
                    
  return frqtemp
    

     


support = 0.005


#将输入数据分区，由于后面要频繁使用。因此缓存起来
#display(dbutils.fs.ls("/FileStore/tables"))
splitedl = sc.textFile('/FileStore/tables/num_1000.txt',8)
#print(input_rdd.collect())


splitedl = splitedl.map(lambda x:splitlines(x))
splitedl.collect()
transationss = splitedl.map(lambda x: x.split(' '))
transationss.collect()
#print('tran:\n',transationss.collect())
#print('spl:\n',splitedl.collect())

numRecords = float(transationss.count())
freqThre = numRecords * support



######
#print('stage:merge')
"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates = transationss.mapPartitions(lambda x: FindFreInPartition(x,support))
#print('globalcan:\n',global_candidates.collect())  
global_candidates = global_candidates.flatMap(lambda its:[frozenset(it) for it in its]).distinct()
#print('globalcan:\n',global_candidates.collect())    

"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
original_data=sc.broadcast(transationss.collect())
    
"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates.map(lambda x:(x,len([transaction for transaction in original_data.value if x.issubset(frozenset(transaction))])/numRecords))\
    .filter(lambda x: x[1]>=support).collectAsMap()
print ('global_frequent_itemsets:\n',global_frequent_itemsets)

  

   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




In [4]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#====================1万测试数据==============
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools
import copy


start = time.clock()
t0 = time.time()



def splitlines(x):
    y = ' '.join(x.split())
    return y
    


def prune(transactions,Ck,support):
    # CSups candidate and supports
    CSups = {}
    for transaction in transactions:
        for candi in Ck:
            candi = frozenset(candi)
            if candi.issubset(frozenset(transaction)):
              if not frozenset(candi) in CSups:
                  CSups[frozenset(candi)] =1
              else:
                  CSups[frozenset(candi)] +=1

                  
    numRecords = float(len(transactions))
    
    # 满足最小支持度的频繁项集和频率
    # FreSups frequent itemset and support
    FreSups = {}
    CurFre = []
    for candi in CSups:
        support_candi = CSups[candi]/numRecords   
        if support_candi >= support:
            CurFre.append(candi)
            FreSups[candi] = support_candi
     
    return CurFre, FreSups
  
def genCk(Lk, k):
    Ck = [] 
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            L1 = list(set(Lk[i]))[: k-2]
            L2 = list(set(Lk[j]))[: k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                Ck.append(frozenset(Lk[i] | Lk[j]))
    return Ck

def FindFreInPartition(iteration,support):
  import itertools
  frqtemp=[]
  preFreSets=[]
  #support = 0.3
  splitedline=list(iteration)
  transactions=copy.deepcopy(splitedline)
  
  numRecords = len(splitedline)
  freqThre = numRecords * support
  #print ("numRecords：" + str(numRecords))
  #print ("frequency threshold：" + str(freqThre))
  #transactions = list(map(lambda x : x.split(' '),splitedline))
                 
         
  #Create candidate_1
  candidate_1 = list(set(list(itertools.chain.from_iterable(transactions))))#distinct
  candidate_1 = list(map(lambda c: [c] ,candidate_1))
  #print(candidate_1)
  CurFre1, FreSups = prune(transactions,candidate_1,support)
  frqtemp.append(frozenset(CurFre1))
  preFreSets = CurFre1
  
         
  #Create CK LK
  K =200 #迭代次数
  for r in range(2,K):
    #print ('round'+str(r)+':\n')
    #print (preFreSets)
    if len(preFreSets)==0:
      break
    candidates = genCk(preFreSets, r)
    CurFre, FreSups = prune(transactions,candidates,support)
    if len(CurFre)!=0:
      frqtemp.append(CurFre)
    preFreSets = CurFre
                    
  return frqtemp
    

     


support = 0.005


#将输入数据分区，由于后面要频繁使用。因此缓存起来
#display(dbutils.fs.ls("/FileStore/tables"))
splitedl = sc.textFile('/FileStore/tables/num_10000.txt',8)
#print(input_rdd.collect())


splitedl = splitedl.map(lambda x:splitlines(x))
splitedl.collect()
transationss = splitedl.map(lambda x: x.split(' '))
transationss.collect()
#print('tran:\n',transationss.collect())
#print('spl:\n',splitedl.collect())

numRecords = float(transationss.count())
freqThre = numRecords * support



######
#print('stage:merge')
"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates = transationss.mapPartitions(lambda x: FindFreInPartition(x,support))
#print('globalcan:\n',global_candidates.collect())  
global_candidates = global_candidates.flatMap(lambda its:[frozenset(it) for it in its]).distinct()
#print('globalcan:\n',global_candidates.collect())    

"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
original_data=sc.broadcast(transationss.collect())
    
"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates.map(lambda x:(x,len([transaction for transaction in original_data.value if x.issubset(frozenset(transaction))])/numRecords))\
    .filter(lambda x: x[1]>=support).collectAsMap()
print ('global_frequent_itemsets:\n',global_frequent_itemsets)

  

   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from collections import OrderedDict
global_frequent_itemsets:
 {frozenset({'1013'}): 0.0055, frozenset({'9579'}): 0.006, frozenset({'136'}): 0.0081, frozenset({'201'}): 0.0153, frozenset({'374'}): 0.0087, frozenset({'178'}): 0.0087, frozenset({'1543'}): 0.008, frozenset({'589'}): 0.014, frozenset({'548'}): 0.0186, frozenset({'840'}): 0.0147, frozenset({'677'}): 0.0151, frozenset({'2626'}): 0.0053, frozenset({'2046'}): 0.0089, frozenset({'76'}): 0.0139, frozenset({'766'}): 0.0098, frozenset({'251'}): 0.0086, frozenset({'3904'}): 0.0176, frozenset({'643'}): 0.0062, frozenset({'479'}): 0.0114, frozenset({'2343'}): 0.0051, frozenset({'170'}): 0.1212, frozenset({'338'}): 0.0168, frozenset({'703'}): 0.0099, frozenset({'1313'}): 0.006, frozenset({'110'}): 0.1023, frozenset({'789'}): 0.0123, frozenset({'1591'}): 0.0055, frozenset({'2505'}): 0.0079, frozenset({'433'}): 0.005, frozenset({'1842'}): 0.0057, frozenset({'681'}): 0.0111, frozenset({'1481'}): 0.0069, frozenset({'529'}): 0.0063, frozenset({'379'}): 0.0097, frozenset({'38', '237'}): 0.0337, frozenset({'38', '48'}): 0.5381, frozenset({'39', '237'}): 0.0466, frozenset({'39', '48'}): 0.8958, frozenset({'237', '32'}): 0.0103, frozenset({'32', '48'}): 0.1965, frozenset({'48', '89'}): 0.0647, frozenset({'107', '38'}): 0.0083, frozenset({'107', '39'}): 0.0138, frozenset({'38', '156'}): 0.01, frozenset({'39', '156'}): 0.0131, frozenset({'1257', '48'}): 0.0058, frozenset({'38', '1188'}): 0.0093, frozenset({'39', '1188'}): 0.0088, frozenset({'179', '48'}): 0.0155, frozenset({'39', '10442'}): 0.007, frozenset({'31', '38'}): 0.0077, frozenset({'31', '39'}): 0.0119, frozenset({'805', '48'}): 0.0053, frozenset({'38', '225'}): 0.0373, frozenset({'39', '225'}): 0.0617, frozenset({'38', '1146'}): 0.0162, frozenset({'39', '1146'}): 0.0316, frozenset({'32', '1146'}): 0.0081, frozenset({'101', '48'}): 0.0283, frozenset({'39', '80'}): 0.0077, frozenset({'39', '984'}): 0.007, frozenset({'264', '48'}): 0.0176, frozenset({'39', '1020'}): 0.0053, frozenset({'55', '48'}): 0.0216, frozenset({'23', '39'}): 0.0104, frozenset({'225', '32'}): 0.015, frozenset({'48', '1831'}): 0.0068, frozenset({'48', '78'}): 0.0186, frozenset({'39', '715'}): 0.0055, frozenset({'38', '389'}): 0.0096, frozenset({'39', '389'}): 0.0148, frozenset({'39', '1003'}): 0.0056, frozenset({'39', '1693'}): 0.0063, frozenset({'56', '38'}): 0.0209, frozenset({'161', '38'}): 0.0085, frozenset({'56', '39'}): 0.0202, frozenset({'161', '39'}): 0.0139, frozenset({'56', '32'}): 0.005, frozenset({'38', '49'}): 0.0124, frozenset({'39', '49'}): 0.0186, frozenset({'32', '49'}): 0.005, frozenset({'1144', '38'}): 0.0053, frozenset({'1144', '39'}): 0.0088, frozenset({'39', '2673'}): 0.0063, frozenset({'38', '488'}): 0.0061, frozenset({'39', '488'}): 0.0109, frozenset({'48', '425'}): 0.008, frozenset({'1727', '48'}): 0.0056, frozenset({'48', '267'}): 0.0107, frozenset({'910', '48'}): 0.0089, frozenset({'2056', '39'}): 0.0065, frozenset({'1239', '48'}): 0.0076, frozenset({'740', '38'}): 0.0082, frozenset({'740', '39'}): 0.0125, frozenset({'718', '39'}): 0.0066, frozenset({'319', '48'}): 0.006, frozenset({'150', '48'}): 0.0075, frozenset({'185', '48'}): 0.0211, frozenset({'522', '38'}): 0.0069, frozenset({'522', '39'}): 0.0131, frozenset({'1355', '48'}): 0.0084, frozenset({'2958', '48'}): 0.0164, frozenset({'38', '3270'}): 0.0103, frozenset({'39', '3270'}): 0.0155, frozenset({'38', '2241'}): 0.0058, frozenset({'39', '2241'}): 0.0085, frozenset({'341', '39'}): 0.0054, frozenset({'396', '48'}): 0.0103, frozenset({'39', '1209'}): 0.0063, frozenset({'19', '48'}): 0.0125, frozenset({'1404', '48'}): 0.0063, frozenset({'38', '1004'}): 0.0072, frozenset({'39', '1004'}): 0.0118, frozenset({'39', '186'}): 0.0095, frozenset({'39', '640'}): 0.0062, frozenset({'39',

In [5]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#====================8万测试数据==============
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools
import copy


start = time.clock()
t0 = time.time()



def splitlines(x):
    y = ' '.join(x.split())
    return y
    


def prune(transactions,Ck,support):
    # CSups candidate and supports
    CSups = {}
    for transaction in transactions:
        for candi in Ck:
            candi = frozenset(candi)
            if candi.issubset(frozenset(transaction)):
              if not frozenset(candi) in CSups:
                  CSups[frozenset(candi)] =1
              else:
                  CSups[frozenset(candi)] +=1

                  
    numRecords = float(len(transactions))
    
    # 满足最小支持度的频繁项集和频率
    # FreSups frequent itemset and support
    FreSups = {}
    CurFre = []
    for candi in CSups:
        support_candi = CSups[candi]/numRecords   
        if support_candi >= support:
            CurFre.append(candi)
            FreSups[candi] = support_candi
     
    return CurFre, FreSups
  
def genCk(Lk, k):
    Ck = [] 
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            L1 = list(set(Lk[i]))[: k-2]
            L2 = list(set(Lk[j]))[: k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                Ck.append(frozenset(Lk[i] | Lk[j]))
    return Ck

def FindFreInPartition(iteration,support):
  import itertools
  frqtemp=[]
  preFreSets=[]
  #support = 0.3
  splitedline=list(iteration)
  transactions=copy.deepcopy(splitedline)
  
  numRecords = len(splitedline)
  freqThre = numRecords * support
  #print ("numRecords：" + str(numRecords))
  #print ("frequency threshold：" + str(freqThre))
  #transactions = list(map(lambda x : x.split(' '),splitedline))
                 
         
  #Create candidate_1
  candidate_1 = list(set(list(itertools.chain.from_iterable(transactions))))#distinct
  candidate_1 = list(map(lambda c: [c] ,candidate_1))
  #print(candidate_1)
  CurFre1, FreSups = prune(transactions,candidate_1,support)
  frqtemp.append(frozenset(CurFre1))
  preFreSets = CurFre1
  
         
  #Create CK LK
  K =200 #迭代次数
  for r in range(2,K):
    #print ('round'+str(r)+':\n')
    #print (preFreSets)
    if len(preFreSets)==0:
      break
    candidates = genCk(preFreSets, r)
    CurFre, FreSups = prune(transactions,candidates,support)
    if len(CurFre)!=0:
      frqtemp.append(CurFre)
    preFreSets = CurFre
                    
  return frqtemp
    

     


support = 0.005


#将输入数据分区，由于后面要频繁使用。因此缓存起来
#display(dbutils.fs.ls("/FileStore/tables"))
splitedl = sc.textFile('/FileStore/tables/retail.txt',8)
#print(input_rdd.collect())


splitedl = splitedl.map(lambda x:splitlines(x))
splitedl.collect()
transationss = splitedl.map(lambda x: x.split(' '))
transationss.collect()
#print('tran:\n',transationss.collect())
#print('spl:\n',splitedl.collect())

numRecords = float(transationss.count())
freqThre = numRecords * support



######
#print('stage:merge')
"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates = transationss.mapPartitions(lambda x: FindFreInPartition(x,support))
#print('globalcan:\n',global_candidates.collect())  
global_candidates = global_candidates.flatMap(lambda its:[frozenset(it) for it in its]).distinct()
#print('globalcan:\n',global_candidates.collect())    

"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
original_data=sc.broadcast(transationss.collect())
    
"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates.map(lambda x:(x,len([transaction for transaction in original_data.value if x.issubset(frozenset(transaction))])/numRecords))\
    .filter(lambda x: x[1]>=support).collectAsMap()
print ('global_frequent_itemsets:\n',global_frequent_itemsets)

  

   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from collections import OrderedDict
global_frequent_itemsets:
 {frozenset({'677'}): 0.012590458474172546, frozenset({'2046'}): 0.007089222113835893, frozenset({'681'}): 0.00798529978902475, frozenset({'53'}): 0.005773462489507952, frozenset({'136'}): 0.005297066763458179, frozenset({'201'}): 0.012851341847961707, frozenset({'374'}): 0.0066241691431682586, frozenset({'76'}): 0.009856854427077425, frozenset({'766'}): 0.00904017603956353, frozenset({'251'}): 0.0070438510923073435, frozenset({'479'}): 0.010503391483859259, frozenset({'1198'}): 0.007486218552210703, frozenset({'170'}): 0.03515119892924389, frozenset({'178'}): 0.005433179828043829, frozenset({'110'}): 0.03169165853769198, frozenset({'789'}): 0.007350105487625054, frozenset({'379'}): 0.007679045393707039, frozenset({'338'}): 0.014450670356843085, frozenset({'703'}): 0.009142260838002767, frozenset({'589'}): 0.012692543272611783, frozenset({'548'}): 0.012896712869490257, frozenset({'39', '389'}): 0.006170458927882761, frozenset({'39', '48'}): 0.33055057734624893, frozenset({'264', '48'}): 0.006057031374061387, frozenset({'38', '48'}): 0.09010684875569973, frozenset({'39', '156'}): 0.006601483632403983, frozenset({'32', '48'}): 0.0911276967400921, frozenset({'39', '237'}): 0.0218801751321431, frozenset({'39', '3270'}): 0.006533427100111159, frozenset({'179', '48'}): 0.007077879358453756, frozenset({'740', '39'}): 0.008609151335042309, frozenset({'1327', '48'}): 0.01097978720990903, frozenset({'48', '78'}): 0.009346430434881241, frozenset({'39', '49'}): 0.008711236133481546, frozenset({'185', '48'}): 0.009255688391824142, frozenset({'23', '39'}): 0.005807490755654364, frozenset({'48', '89'}): 0.031737029559220524, frozenset({'19', '48'}): 0.0067035684308432205, frozenset({'101', '48'}): 0.01487035230598217, frozenset({'39', '225'}): 0.026666817903405096, frozenset({'56', '38'}): 0.005830176266418638, frozenset({'39', '1146'}): 0.011149928540641093, frozenset({'39', '1004'}): 0.006612826387786121, frozenset({'225', '32'}): 0.007429504775300016, frozenset({'161', '39'}): 0.007055193847689481, frozenset({'2958', '48'}): 0.008836006442685057, frozenset({'38', '237'}): 0.007747101925999864, frozenset({'38', '225'}): 0.007724416415235589, frozenset({'31', '39'}): 0.0061024023955899365, frozenset({'237', '32'}): 0.0064426850570540595, frozenset({'522', '39'}): 0.007350105487625054, frozenset({'39', '258', '48'}): 0.005126925432726118, frozenset({'310', '39', '48'}): 0.015278691499739117, frozenset({'592', '39', '48'}): 0.0058415190218007755, frozenset({'39', '48', '413'}): 0.008858691953449332, frozenset({'39', '2238', '48'}): 0.008938091241124295, frozenset({'286', '39', '38'}): 0.008257525918196049, frozenset({'475', '39', '48'}): 0.012386288877294073, frozenset({'301', '39', '48'}): 0.005365123295751004, frozenset({'286', '38', '39', '48'}): 0.005194981965018943, frozenset({'10', '39'}): 0.005126925432726118, frozenset({'271', '41'}): 0.0066241691431682586, frozenset({'175', '48'}): 0.0055919784033937525, frozenset({'170', '48', '41'}): 0.005580635648011615, frozenset({'38', '39', '170', '41'}): 0.006975794560014519, frozenset({'38', '39', '110', '41'}): 0.005796148000272226, frozenset({'39', '806'}): 0.0055919784033937525, frozenset({'12929'}): 0.008416324493545973, frozenset({'15618'}): 0.0069644518046323815, frozenset({'16010', '39', '48'}): 0.0060003175971507, frozenset({'1783'}): 0.006805653229282458, frozenset({'107'}): 0.008677207867335134, frozenset({'161'}): 0.011456182935958802, frozenset({'1020'}): 0.005274381252693905, frozenset({'488'}): 0.00816678387513895, frozenset({'31'}): 0.010435334951566434, frozenset({'94'}): 0.0058528617771829135, frozenset({'740'}): 0.013395794106304303, frozenset({'1146'}): 0.016174769174927973, frozenset({'522'}): 0.01104784374220

In [6]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#====================8万测试数据,分区：4==============
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools
import copy


start = time.clock()
t0 = time.time()



def splitlines(x):
    y = ' '.join(x.split())
    return y
    


def prune(transactions,Ck,support):
    # CSups candidate and supports
    CSups = {}
    for transaction in transactions:
        for candi in Ck:
            candi = frozenset(candi)
            if candi.issubset(frozenset(transaction)):
              if not frozenset(candi) in CSups:
                  CSups[frozenset(candi)] =1
              else:
                  CSups[frozenset(candi)] +=1

                  
    numRecords = float(len(transactions))
    
    # 满足最小支持度的频繁项集和频率
    # FreSups frequent itemset and support
    FreSups = {}
    CurFre = []
    for candi in CSups:
        support_candi = CSups[candi]/numRecords   
        if support_candi >= support:
            CurFre.append(candi)
            FreSups[candi] = support_candi
     
    return CurFre, FreSups
  
def genCk(Lk, k):
    Ck = [] 
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            L1 = list(set(Lk[i]))[: k-2]
            L2 = list(set(Lk[j]))[: k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                Ck.append(frozenset(Lk[i] | Lk[j]))
    return Ck

def FindFreInPartition(iteration,support):
  import itertools
  frqtemp=[]
  preFreSets=[]
  #support = 0.3
  splitedline=list(iteration)
  transactions=copy.deepcopy(splitedline)
  
  numRecords = len(splitedline)
  freqThre = numRecords * support
  #print ("numRecords：" + str(numRecords))
  #print ("frequency threshold：" + str(freqThre))
  #transactions = list(map(lambda x : x.split(' '),splitedline))
                 
         
  #Create candidate_1
  candidate_1 = list(set(list(itertools.chain.from_iterable(transactions))))#distinct
  candidate_1 = list(map(lambda c: [c] ,candidate_1))
  #print(candidate_1)
  CurFre1, FreSups = prune(transactions,candidate_1,support)
  frqtemp.append(frozenset(CurFre1))
  preFreSets = CurFre1
  
         
  #Create CK LK
  K =200 #迭代次数
  for r in range(2,K):
    #print ('round'+str(r)+':\n')
    #print (preFreSets)
    if len(preFreSets)==0:
      break
    candidates = genCk(preFreSets, r)
    CurFre, FreSups = prune(transactions,candidates,support)
    if len(CurFre)!=0:
      frqtemp.append(CurFre)
    preFreSets = CurFre
                    
  return frqtemp
    

     


support = 0.005


#将输入数据分区，由于后面要频繁使用。因此缓存起来
#display(dbutils.fs.ls("/FileStore/tables"))
splitedl = sc.textFile('/FileStore/tables/retail.txt',4)
#print(input_rdd.collect())


splitedl = splitedl.map(lambda x:splitlines(x))
splitedl.collect()
transationss = splitedl.map(lambda x: x.split(' '))
transationss.collect()
#print('tran:\n',transationss.collect())
#print('spl:\n',splitedl.collect())

numRecords = float(transationss.count())
freqThre = numRecords * support



######
#print('stage:merge')
"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates = transationss.mapPartitions(lambda x: FindFreInPartition(x,support))
#print('globalcan:\n',global_candidates.collect())  
global_candidates = global_candidates.flatMap(lambda its:[frozenset(it) for it in its]).distinct()
#print('globalcan:\n',global_candidates.collect())    

"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
original_data=sc.broadcast(transationss.collect())
    
"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates.map(lambda x:(x,len([transaction for transaction in original_data.value if x.issubset(frozenset(transaction))])/numRecords))\
    .filter(lambda x: x[1]>=support).collectAsMap()
print ('global_frequent_itemsets:\n',global_frequent_itemsets)

  

   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from collections import OrderedDict
global_frequent_itemsets:
 {frozenset({'677'}): 0.012590458474172546, frozenset({'425'}): 0.0068737097615752816, frozenset({'2046'}): 0.007089222113835893, frozenset({'681'}): 0.00798529978902475, frozenset({'89'}): 0.04352215240126132, frozenset({'53'}): 0.005773462489507952, frozenset({'32'}): 0.1720355708808784, frozenset({'38'}): 0.17690161293981535, frozenset({'201'}): 0.012851341847961707, frozenset({'910'}): 0.005886890043329326, frozenset({'2958'}): 0.010253850865452236, frozenset({'76'}): 0.009856854427077425, frozenset({'374'}): 0.0066241691431682586, frozenset({'251'}): 0.0070438510923073435, frozenset({'766'}): 0.00904017603956353, frozenset({'1239'}): 0.006454027812436197, frozenset({'479'}): 0.010503391483859259, frozenset({'267'}): 0.0066241691431682586, frozenset({'150'}): 0.006771624963136045, frozenset({'1198'}): 0.007486218552210703, frozenset({'179'}): 0.011320069871373154, frozenset({'170'}): 0.03515119892924389, frozenset({'178'}): 0.005433179828043829, frozenset({'55'}): 0.00798529978902475, frozenset({'110'}): 0.03169165853769198, frozenset({'675'}): 0.006816995984664595, frozenset({'789'}): 0.007350105487625054, frozenset({'379'}): 0.007679045393707039, frozenset({'264'}): 0.010151766067012999, frozenset({'338'}): 0.014450670356843085, frozenset({'961'}): 0.006125087906354212, frozenset({'1355'}): 0.005807490755654364, frozenset({'703'}): 0.009142260838002767, frozenset({'589'}): 0.012692543272611783, frozenset({'185'}): 0.015607631405821103, frozenset({'19'}): 0.011399469159048116, frozenset({'39'}): 0.5747941289898142, frozenset({'1435'}): 0.008677207867335134, frozenset({'548'}): 0.012896712869490257, frozenset({'78'}): 0.012023320705065675, frozenset({'101'}): 0.02537374378984143, frozenset({'1327'}): 0.02025816111249745, frozenset({'396'}): 0.007645017127560627, frozenset({'175'}): 0.011002472720673307, frozenset({'38', '48'}): 0.09010684875569973, frozenset({'39', '48'}): 0.33055057734624893, frozenset({'101', '48'}): 0.01487035230598217, frozenset({'48', '89'}): 0.031737029559220524, frozenset({'39', '117'}): 0.006283886481704136, frozenset({'170', '48'}): 0.017660670129987975, frozenset({'32', '48'}): 0.0911276967400921, frozenset({'179', '48'}): 0.007077879358453756, frozenset({'38', '237'}): 0.007747101925999864, frozenset({'39', '237'}): 0.0218801751321431, frozenset({'48', '78'}): 0.009346430434881241, frozenset({'38', '225'}): 0.007724416415235589, frozenset({'39', '225'}): 0.026666817903405096, frozenset({'270', '39'}): 0.01354324992627209, frozenset({'38', '105'}): 0.0072933917107143665, frozenset({'161', '39'}): 0.007055193847689481, frozenset({'338', '48'}): 0.008836006442685057, frozenset({'237', '32'}): 0.0064426850570540595, frozenset({'264', '48'}): 0.006057031374061387, frozenset({'548', '48'}): 0.007463533041446428, frozenset({'589', '48'}): 0.007123250379982305, frozenset({'110', '48'}): 0.01565300242734965, frozenset({'201', '48'}): 0.007645017127560627, frozenset({'23', '39'}): 0.005807490755654364, frozenset({'39', '533'}): 0.010458020462330709, frozenset({'48', '19'}): 0.0067035684308432205, frozenset({'740', '39'}): 0.008609151335042309, frozenset({'39', '812'}): 0.005342437784986729, frozenset({'522', '39'}): 0.007350105487625054, frozenset({'39', '1146'}): 0.011149928540641093, frozenset({'677', '48'}): 0.0075542750845035275, frozenset({'271', '41'}): 0.0066241691431682586, frozenset({'1327', '48'}): 0.01097978720990903, frozenset({'334', '39'}): 0.005739434223361539, frozenset({'39', '1004'}): 0.006612826387786121, frozenset({'1814', '39'}): 0.005342437784986729, frozenset({'39', '976'}): 0.005342437784986729, frozenset({'185', '48'}): 0.009255688391824142, frozenset({'39', '156'}): 0.006601483632403983, frozenset({'225', '32'}): 0.0

In [7]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#====================8万测试数据,分区：8==============
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools
import copy


start = time.clock()
t0 = time.time()



def splitlines(x):
    y = ' '.join(x.split())
    return y
    


def prune(transactions,Ck,support):
    # CSups candidate and supports
    CSups = {}
    for transaction in transactions:
        for candi in Ck:
            candi = frozenset(candi)
            if candi.issubset(frozenset(transaction)):
              if not frozenset(candi) in CSups:
                  CSups[frozenset(candi)] =1
              else:
                  CSups[frozenset(candi)] +=1

                  
    numRecords = float(len(transactions))
    
    # 满足最小支持度的频繁项集和频率
    # FreSups frequent itemset and support
    FreSups = {}
    CurFre = []
    for candi in CSups:
        support_candi = CSups[candi]/numRecords   
        if support_candi >= support:
            CurFre.append(candi)
            FreSups[candi] = support_candi
     
    return CurFre, FreSups
  
def genCk(Lk, k):
    Ck = [] 
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            L1 = list(set(Lk[i]))[: k-2]
            L2 = list(set(Lk[j]))[: k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                Ck.append(frozenset(Lk[i] | Lk[j]))
    return Ck

def FindFreInPartition(iteration,support):
  import itertools
  frqtemp=[]
  preFreSets=[]
  #support = 0.3
  splitedline=list(iteration)
  transactions=copy.deepcopy(splitedline)
  
  numRecords = len(splitedline)
  freqThre = numRecords * support
  #print ("numRecords：" + str(numRecords))
  #print ("frequency threshold：" + str(freqThre))
  #transactions = list(map(lambda x : x.split(' '),splitedline))
                 
         
  #Create candidate_1
  candidate_1 = list(set(list(itertools.chain.from_iterable(transactions))))#distinct
  candidate_1 = list(map(lambda c: [c] ,candidate_1))
  #print(candidate_1)
  CurFre1, FreSups = prune(transactions,candidate_1,support)
  frqtemp.append(frozenset(CurFre1))
  preFreSets = CurFre1
  
         
  #Create CK LK
  K =200 #迭代次数
  for r in range(2,K):
    #print ('round'+str(r)+':\n')
    #print (preFreSets)
    if len(preFreSets)==0:
      break
    candidates = genCk(preFreSets, r)
    CurFre, FreSups = prune(transactions,candidates,support)
    if len(CurFre)!=0:
      frqtemp.append(CurFre)
    preFreSets = CurFre
                    
  return frqtemp
    

     


support = 0.005


#将输入数据分区，由于后面要频繁使用。因此缓存起来
#display(dbutils.fs.ls("/FileStore/tables"))
splitedl = sc.textFile('/FileStore/tables/retail.txt',8)
#print(input_rdd.collect())


splitedl = splitedl.map(lambda x:splitlines(x))
splitedl.collect()
transationss = splitedl.map(lambda x: x.split(' '))
transationss.collect()
#print('tran:\n',transationss.collect())
#print('spl:\n',splitedl.collect())

numRecords = float(transationss.count())
freqThre = numRecords * support



######
#print('stage:merge')
"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates = transationss.mapPartitions(lambda x: FindFreInPartition(x,support))
#print('globalcan:\n',global_candidates.collect())  
global_candidates = global_candidates.flatMap(lambda its:[frozenset(it) for it in its]).distinct()
#print('globalcan:\n',global_candidates.collect())    

"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
original_data=sc.broadcast(transationss.collect())
    
"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates.map(lambda x:(x,len([transaction for transaction in original_data.value if x.issubset(frozenset(transaction))])/numRecords))\
    .filter(lambda x: x[1]>=support).collectAsMap()
print ('global_frequent_itemsets:\n',global_frequent_itemsets)

  

   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from collections import OrderedDict
global_frequent_itemsets:
 {frozenset({'677'}): 0.012590458474172546, frozenset({'2046'}): 0.007089222113835893, frozenset({'681'}): 0.00798529978902475, frozenset({'53'}): 0.005773462489507952, frozenset({'136'}): 0.005297066763458179, frozenset({'201'}): 0.012851341847961707, frozenset({'374'}): 0.0066241691431682586, frozenset({'76'}): 0.009856854427077425, frozenset({'766'}): 0.00904017603956353, frozenset({'251'}): 0.0070438510923073435, frozenset({'479'}): 0.010503391483859259, frozenset({'1198'}): 0.007486218552210703, frozenset({'170'}): 0.03515119892924389, frozenset({'178'}): 0.005433179828043829, frozenset({'110'}): 0.03169165853769198, frozenset({'789'}): 0.007350105487625054, frozenset({'379'}): 0.007679045393707039, frozenset({'338'}): 0.014450670356843085, frozenset({'703'}): 0.009142260838002767, frozenset({'589'}): 0.012692543272611783, frozenset({'548'}): 0.012896712869490257, frozenset({'39', '389'}): 0.006170458927882761, frozenset({'39', '48'}): 0.33055057734624893, frozenset({'264', '48'}): 0.006057031374061387, frozenset({'38', '48'}): 0.09010684875569973, frozenset({'39', '156'}): 0.006601483632403983, frozenset({'32', '48'}): 0.0911276967400921, frozenset({'39', '237'}): 0.0218801751321431, frozenset({'39', '3270'}): 0.006533427100111159, frozenset({'179', '48'}): 0.007077879358453756, frozenset({'740', '39'}): 0.008609151335042309, frozenset({'1327', '48'}): 0.01097978720990903, frozenset({'48', '78'}): 0.009346430434881241, frozenset({'39', '49'}): 0.008711236133481546, frozenset({'185', '48'}): 0.009255688391824142, frozenset({'23', '39'}): 0.005807490755654364, frozenset({'48', '89'}): 0.031737029559220524, frozenset({'19', '48'}): 0.0067035684308432205, frozenset({'101', '48'}): 0.01487035230598217, frozenset({'39', '225'}): 0.026666817903405096, frozenset({'56', '38'}): 0.005830176266418638, frozenset({'39', '1146'}): 0.011149928540641093, frozenset({'39', '1004'}): 0.006612826387786121, frozenset({'225', '32'}): 0.007429504775300016, frozenset({'161', '39'}): 0.007055193847689481, frozenset({'2958', '48'}): 0.008836006442685057, frozenset({'38', '237'}): 0.007747101925999864, frozenset({'38', '225'}): 0.007724416415235589, frozenset({'31', '39'}): 0.0061024023955899365, frozenset({'237', '32'}): 0.0064426850570540595, frozenset({'522', '39'}): 0.007350105487625054, frozenset({'39', '258', '48'}): 0.005126925432726118, frozenset({'310', '39', '48'}): 0.015278691499739117, frozenset({'592', '39', '48'}): 0.0058415190218007755, frozenset({'39', '48', '413'}): 0.008858691953449332, frozenset({'39', '2238', '48'}): 0.008938091241124295, frozenset({'286', '39', '38'}): 0.008257525918196049, frozenset({'475', '39', '48'}): 0.012386288877294073, frozenset({'301', '39', '48'}): 0.005365123295751004, frozenset({'286', '38', '39', '48'}): 0.005194981965018943, frozenset({'10', '39'}): 0.005126925432726118, frozenset({'271', '41'}): 0.0066241691431682586, frozenset({'175', '48'}): 0.0055919784033937525, frozenset({'39', '806'}): 0.0055919784033937525, frozenset({'170', '48', '41'}): 0.005580635648011615, frozenset({'38', '39', '170', '41'}): 0.006975794560014519, frozenset({'38', '39', '110', '41'}): 0.005796148000272226, frozenset({'15618'}): 0.0069644518046323815, frozenset({'12929'}): 0.008416324493545973, frozenset({'16010', '39', '48'}): 0.0060003175971507, frozenset({'1783'}): 0.006805653229282458, frozenset({'107'}): 0.008677207867335134, frozenset({'161'}): 0.011456182935958802, frozenset({'1020'}): 0.005274381252693905, frozenset({'488'}): 0.00816678387513895, frozenset({'31'}): 0.010435334951566434, frozenset({'94'}): 0.0058528617771829135, frozenset({'740'}): 0.013395794106304303, frozenset({'1146'}): 0.016174769174927973, frozenset({'522'}): 0.01104784374220

In [8]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#====================8万测试数据,分区：12==============
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools
import copy


start = time.clock()
t0 = time.time()



def splitlines(x):
    y = ' '.join(x.split())
    return y
    


def prune(transactions,Ck,support):
    # CSups candidate and supports
    CSups = {}
    for transaction in transactions:
        for candi in Ck:
            candi = frozenset(candi)
            if candi.issubset(frozenset(transaction)):
              if not frozenset(candi) in CSups:
                  CSups[frozenset(candi)] =1
              else:
                  CSups[frozenset(candi)] +=1

                  
    numRecords = float(len(transactions))
    
    # 满足最小支持度的频繁项集和频率
    # FreSups frequent itemset and support
    FreSups = {}
    CurFre = []
    for candi in CSups:
        support_candi = CSups[candi]/numRecords   
        if support_candi >= support:
            CurFre.append(candi)
            FreSups[candi] = support_candi
     
    return CurFre, FreSups
  
def genCk(Lk, k):
    Ck = [] 
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            L1 = list(set(Lk[i]))[: k-2]
            L2 = list(set(Lk[j]))[: k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                Ck.append(frozenset(Lk[i] | Lk[j]))
    return Ck

def FindFreInPartition(iteration,support):
  import itertools
  frqtemp=[]
  preFreSets=[]
  #support = 0.3
  splitedline=list(iteration)
  transactions=copy.deepcopy(splitedline)
  
  numRecords = len(splitedline)
  freqThre = numRecords * support
  #print ("numRecords：" + str(numRecords))
  #print ("frequency threshold：" + str(freqThre))
  #transactions = list(map(lambda x : x.split(' '),splitedline))
                 
         
  #Create candidate_1
  candidate_1 = list(set(list(itertools.chain.from_iterable(transactions))))#distinct
  candidate_1 = list(map(lambda c: [c] ,candidate_1))
  #print(candidate_1)
  CurFre1, FreSups = prune(transactions,candidate_1,support)
  frqtemp.append(frozenset(CurFre1))
  preFreSets = CurFre1
  
         
  #Create CK LK
  K =200 #迭代次数
  for r in range(2,K):
    #print ('round'+str(r)+':\n')
    #print (preFreSets)
    if len(preFreSets)==0:
      break
    candidates = genCk(preFreSets, r)
    CurFre, FreSups = prune(transactions,candidates,support)
    if len(CurFre)!=0:
      frqtemp.append(CurFre)
    preFreSets = CurFre
                    
  return frqtemp
    

     


support = 0.005


#将输入数据分区，由于后面要频繁使用。因此缓存起来
#display(dbutils.fs.ls("/FileStore/tables"))
splitedl = sc.textFile('/FileStore/tables/retail.txt',12)
#print(input_rdd.collect())


splitedl = splitedl.map(lambda x:splitlines(x))
splitedl.collect()
transationss = splitedl.map(lambda x: x.split(' '))
transationss.collect()
#print('tran:\n',transationss.collect())
#print('spl:\n',splitedl.collect())

numRecords = float(transationss.count())
freqThre = numRecords * support



######
#print('stage:merge')
"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates = transationss.mapPartitions(lambda x: FindFreInPartition(x,support))
#print('globalcan:\n',global_candidates.collect())  
global_candidates = global_candidates.flatMap(lambda its:[frozenset(it) for it in its]).distinct()
#print('globalcan:\n',global_candidates.collect())    

"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
original_data=sc.broadcast(transationss.collect())
    
"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates.map(lambda x:(x,len([transaction for transaction in original_data.value if x.issubset(frozenset(transaction))])/numRecords))\
    .filter(lambda x: x[1]>=support).collectAsMap()
print ('global_frequent_itemsets:\n',global_frequent_itemsets)

  

   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from collections import OrderedDict
global_frequent_itemsets:
 {frozenset({'677'}): 0.012590458474172546, frozenset({'136'}): 0.005297066763458179, frozenset({'38'}): 0.17690161293981535, frozenset({'201'}): 0.012851341847961707, frozenset({'2958'}): 0.010253850865452236, frozenset({'76'}): 0.009856854427077425, frozenset({'1239'}): 0.006454027812436197, frozenset({'12929'}): 0.008416324493545973, frozenset({'267'}): 0.0066241691431682586, frozenset({'150'}): 0.006771624963136045, frozenset({'789'}): 0.007350105487625054, frozenset({'379'}): 0.007679045393707039, frozenset({'264'}): 0.010151766067012999, frozenset({'961'}): 0.006125087906354212, frozenset({'19'}): 0.011399469159048116, frozenset({'39'}): 0.5747941289898142, frozenset({'548'}): 0.012896712869490257, frozenset({'1327'}): 0.02025816111249745, frozenset({'175'}): 0.011002472720673307, frozenset({'38', '225'}): 0.007724416415235589, frozenset({'56', '38'}): 0.005830176266418638, frozenset({'39', '225'}): 0.026666817903405096, frozenset({'39', '533'}): 0.010458020462330709, frozenset({'48', '89'}): 0.031737029559220524, frozenset({'19', '48'}): 0.0067035684308432205, frozenset({'39', '269'}): 0.0050021551235226065, frozenset({'39', '105'}): 0.00516095369887253, frozenset({'39', '237'}): 0.0218801751321431, frozenset({'110', '48'}): 0.01565300242734965, frozenset({'39', '1004'}): 0.006612826387786121, frozenset({'201', '48'}): 0.007645017127560627, frozenset({'264', '48'}): 0.006057031374061387, frozenset({'334', '39'}): 0.005739434223361539, frozenset({'479', '48'}): 0.005818833511036501, frozenset({'179', '48'}): 0.007077879358453756, frozenset({'39', '156'}): 0.006601483632403983, frozenset({'1327', '48'}): 0.01097978720990903, frozenset({'475', '39', '48'}): 0.012386288877294073, frozenset({'65', '39', '48'}): 0.02038293142170096, frozenset({'38', '36', '48'}): 0.015426147319706903, frozenset({'286', '39', '38'}): 0.008257525918196049, frozenset({'39', '255', '48'}): 0.00922166012567773, frozenset({'286', '38', '39', '48'}): 0.005194981965018943, frozenset({'178'}): 0.005433179828043829, frozenset({'48', '76'}): 0.005501236360336653, frozenset({'10', '39'}): 0.005126925432726118, frozenset({'38', '48', '41'}): 0.026927701277194255, frozenset({'170', '48', '41'}): 0.005580635648011615, frozenset({'32', '48', '41'}): 0.023400104353349514, frozenset({'39', '258', '48'}): 0.005126925432726118, frozenset({'65', '32', '48'}): 0.005580635648011615, frozenset({'38', '39', '170', '41'}): 0.006975794560014519, frozenset({'16010', '39', '48'}): 0.0060003175971507, frozenset({'10446'}): 0.007009822826160931, frozenset({'175', '48'}): 0.0055919784033937525, frozenset({'1693'}): 0.005943603820240013, frozenset({'581'}): 0.006510741589346884, frozenset({'161'}): 0.011456182935958802, frozenset({'488'}): 0.00816678387513895, frozenset({'740'}): 0.013395794106304303, frozenset({'10'}): 0.008076041832081849, frozenset({'522'}): 0.011047843742201855, frozenset({'225'}): 0.03694335427962161, frozenset({'885'}): 0.006533427100111159, frozenset({'2329'}): 0.005535264626483065, frozenset({'976'}): 0.009267031147206279, frozenset({'345'}): 0.00555795013724734, frozenset({'156'}): 0.009414486967174066, frozenset({'105'}): 0.00745219028606429, frozenset({'52'}): 0.005682720446450852, frozenset({'2399'}): 0.005455865338808103, frozenset({'1814'}): 0.007803815702910551, frozenset({'270', '48'}): 0.01085501690070552, frozenset({'39', '38'}): 0.1173408044282117, frozenset({'38', '32'}): 0.032134025997595336, frozenset({'39', '32'}): 0.09590299675597196, frozenset({'39', '101'}): 0.015879857534992402, frozenset({'39', '589'}): 0.0080306708105533, frozenset({'264', '39'}): 0.00679431047390032, frozenset({'110', '32'}): 0.005092897166579706, frozenset({'740', '48'}): 0.005898232798711463, froz

In [9]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#====================8万测试数据,分区：16==============
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools
import copy


start = time.clock()
t0 = time.time()



def splitlines(x):
    y = ' '.join(x.split())
    return y
    


def prune(transactions,Ck,support):
    # CSups candidate and supports
    CSups = {}
    for transaction in transactions:
        for candi in Ck:
            candi = frozenset(candi)
            if candi.issubset(frozenset(transaction)):
              if not frozenset(candi) in CSups:
                  CSups[frozenset(candi)] =1
              else:
                  CSups[frozenset(candi)] +=1

                  
    numRecords = float(len(transactions))
    
    # 满足最小支持度的频繁项集和频率
    # FreSups frequent itemset and support
    FreSups = {}
    CurFre = []
    for candi in CSups:
        support_candi = CSups[candi]/numRecords   
        if support_candi >= support:
            CurFre.append(candi)
            FreSups[candi] = support_candi
     
    return CurFre, FreSups
  
def genCk(Lk, k):
    Ck = [] 
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            L1 = list(set(Lk[i]))[: k-2]
            L2 = list(set(Lk[j]))[: k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                Ck.append(frozenset(Lk[i] | Lk[j]))
    return Ck

def FindFreInPartition(iteration,support):
  import itertools
  frqtemp=[]
  preFreSets=[]
  #support = 0.3
  splitedline=list(iteration)
  transactions=copy.deepcopy(splitedline)
  
  numRecords = len(splitedline)
  freqThre = numRecords * support
  #print ("numRecords：" + str(numRecords))
  #print ("frequency threshold：" + str(freqThre))
  #transactions = list(map(lambda x : x.split(' '),splitedline))
                 
         
  #Create candidate_1
  candidate_1 = list(set(list(itertools.chain.from_iterable(transactions))))#distinct
  candidate_1 = list(map(lambda c: [c] ,candidate_1))
  #print(candidate_1)
  CurFre1, FreSups = prune(transactions,candidate_1,support)
  frqtemp.append(frozenset(CurFre1))
  preFreSets = CurFre1
  
         
  #Create CK LK
  K =200 #迭代次数
  for r in range(2,K):
    #print ('round'+str(r)+':\n')
    #print (preFreSets)
    if len(preFreSets)==0:
      break
    candidates = genCk(preFreSets, r)
    CurFre, FreSups = prune(transactions,candidates,support)
    if len(CurFre)!=0:
      frqtemp.append(CurFre)
    preFreSets = CurFre
                    
  return frqtemp
    

     


support = 0.005


#将输入数据分区，由于后面要频繁使用。因此缓存起来
#display(dbutils.fs.ls("/FileStore/tables"))
splitedl = sc.textFile('/FileStore/tables/retail.txt',16)
#print(input_rdd.collect())


splitedl = splitedl.map(lambda x:splitlines(x))
splitedl.collect()
transationss = splitedl.map(lambda x: x.split(' '))
transationss.collect()
#print('tran:\n',transationss.collect())
#print('spl:\n',splitedl.collect())

numRecords = float(transationss.count())
freqThre = numRecords * support



######
#print('stage:merge')
"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates = transationss.mapPartitions(lambda x: FindFreInPartition(x,support))
#print('globalcan:\n',global_candidates.collect())  
global_candidates = global_candidates.flatMap(lambda its:[frozenset(it) for it in its]).distinct()
#print('globalcan:\n',global_candidates.collect())    

"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
original_data=sc.broadcast(transationss.collect())
    
"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates.map(lambda x:(x,len([transaction for transaction in original_data.value if x.issubset(frozenset(transaction))])/numRecords))\
    .filter(lambda x: x[1]>=support).collectAsMap()
print ('global_frequent_itemsets:\n',global_frequent_itemsets)

  

   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from collections import OrderedDict
global_frequent_itemsets:
 {frozenset({'681'}): 0.00798529978902475, frozenset({'201'}): 0.012851341847961707, frozenset({'374'}): 0.0066241691431682586, frozenset({'76'}): 0.009856854427077425, frozenset({'251'}): 0.0070438510923073435, frozenset({'766'}): 0.00904017603956353, frozenset({'479'}): 0.010503391483859259, frozenset({'1198'}): 0.007486218552210703, frozenset({'170'}): 0.03515119892924389, frozenset({'110'}): 0.03169165853769198, frozenset({'789'}): 0.007350105487625054, frozenset({'338'}): 0.014450670356843085, frozenset({'589'}): 0.012692543272611783, frozenset({'548'}): 0.012896712869490257, frozenset({'19', '48'}): 0.0067035684308432205, frozenset({'38', '48'}): 0.09010684875569973, frozenset({'39', '237'}): 0.0218801751321431, frozenset({'237', '32'}): 0.0064426850570540595, frozenset({'39', '1004'}): 0.006612826387786121, frozenset({'522', '39'}): 0.007350105487625054, frozenset({'48', '89'}): 0.031737029559220524, frozenset({'23', '39'}): 0.005807490755654364, frozenset({'38', '225'}): 0.007724416415235589, frozenset({'161', '39'}): 0.007055193847689481, frozenset({'31', '39'}): 0.0061024023955899365, frozenset({'310', '39', '48'}): 0.015278691499739117, frozenset({'286', '38', '39'}): 0.008257525918196049, frozenset({'301', '39', '48'}): 0.005365123295751004, frozenset({'39', '2238', '48'}): 0.008938091241124295, frozenset({'136'}): 0.005297066763458179, frozenset({'178'}): 0.005433179828043829, frozenset({'56', '38'}): 0.005830176266418638, frozenset({'48', '78'}): 0.009346430434881241, frozenset({'592', '39', '48'}): 0.0058415190218007755, frozenset({'39', '156'}): 0.006601483632403983, frozenset({'39', '3270'}): 0.006533427100111159, frozenset({'10', '39'}): 0.005126925432726118, frozenset({'53'}): 0.005773462489507952, frozenset({'175', '48'}): 0.0055919784033937525, frozenset({'16010', '39', '48'}): 0.0060003175971507, frozenset({'107'}): 0.008677207867335134, frozenset({'488'}): 0.00816678387513895, frozenset({'740'}): 0.013395794106304303, frozenset({'1146'}): 0.016174769174927973, frozenset({'56'}): 0.006068374129443524, frozenset({'225'}): 0.03694335427962161, frozenset({'389'}): 0.009425829722556203, frozenset({'1986'}): 0.0069644518046323815, frozenset({'48'}): 0.47792699802636057, frozenset({'49'}): 0.01270388602799392, frozenset({'209'}): 0.006930423538485969, frozenset({'39', '12929'}): 0.0053310950296045915, frozenset({'703', '39'}): 0.005138268188108255, frozenset({'38', '110'}): 0.030909008416324494, frozenset({'334', '48'}): 0.005024840634286881, frozenset({'38', '170'}): 0.03437989156325855, frozenset({'677', '39'}): 0.007202649667657267, frozenset({'38', '37', '48'}): 0.006317914747850548, frozenset({'65', '38', '39'}): 0.005535264626483065, frozenset({'438', '39', '48'}): 0.008847349198067194, frozenset({'39', '38', '36', '48'}): 0.012250175812708423, frozenset({'186'}): 0.007486218552210703, frozenset({'806'}): 0.008404981738163834, frozenset({'310', '41'}): 0.008155441119756811, frozenset({'2238', '41'}): 0.006465370567818335, frozenset({'38', '39', '41'}): 0.034606746670901294, frozenset({'89', '39', '41'}): 0.007021165581543068, frozenset({'249', '39', '48'}): 0.005784805244890089, frozenset({'38', '32', '41'}): 0.00913091808262063, frozenset({'39', '32', '48', '41'}): 0.018670175358998207, frozenset({'1693'}): 0.005943603820240013, frozenset({'16010', '41'}): 0.005297066763458179, frozenset({'432'}): 0.005285724008076042, frozenset({'147'}): 0.020178761824822485, frozenset({'208'}): 0.0060003175971507, frozenset({'41'}): 0.16951747918604387, frozenset({'407'}): 0.007860529479821238, frozenset({'420'}): 0.006805653229282458, frozenset({'18'}): 0.009754769628638188, frozenset({'65'}): 0.05072480206891858, frozenset({'60'}): 0.01688936276400263, f

In [10]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#====================w=4==============
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools
import copy


start = time.clock()
t0 = time.time()



def splitlines(x):
    y = ' '.join(x.split())
    return y
    


def prune(transactions,Ck,support):
    # CSups candidate and supports
    CSups = {}
    for transaction in transactions:
        for candi in Ck:
            candi = frozenset(candi)
            if candi.issubset(frozenset(transaction)):
              if not frozenset(candi) in CSups:
                  CSups[frozenset(candi)] =1
              else:
                  CSups[frozenset(candi)] +=1

                  
    numRecords = float(len(transactions))
    
    # 满足最小支持度的频繁项集和频率
    # FreSups frequent itemset and support
    FreSups = {}
    CurFre = []
    for candi in CSups:
        support_candi = CSups[candi]/numRecords   
        if support_candi >= support:
            CurFre.append(candi)
            FreSups[candi] = support_candi
     
    return CurFre, FreSups
  
def genCk(Lk, k):
    Ck = [] 
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            L1 = list(set(Lk[i]))[: k-2]
            L2 = list(set(Lk[j]))[: k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                Ck.append(frozenset(Lk[i] | Lk[j]))
    return Ck

def FindFreInPartition(iteration,support):
  import itertools
  frqtemp=[]
  preFreSets=[]
  #support = 0.3
  splitedline=list(iteration)
  transactions=copy.deepcopy(splitedline)
  
  numRecords = len(splitedline)
  freqThre = numRecords * support
  #print ("numRecords：" + str(numRecords))
  #print ("frequency threshold：" + str(freqThre))
  #transactions = list(map(lambda x : x.split(' '),splitedline))
                 
         
  #Create candidate_1
  candidate_1 = list(set(list(itertools.chain.from_iterable(transactions))))#distinct
  candidate_1 = list(map(lambda c: [c] ,candidate_1))
  #print(candidate_1)
  CurFre1, FreSups = prune(transactions,candidate_1,support)
  frqtemp.append(frozenset(CurFre1))
  preFreSets = CurFre1
  
         
  #Create CK LK
  K =200 #迭代次数
  for r in range(2,K):
    #print ('round'+str(r)+':\n')
    #print (preFreSets)
    if len(preFreSets)==0:
      break
    candidates = genCk(preFreSets, r)
    CurFre, FreSups = prune(transactions,candidates,support)
    if len(CurFre)!=0:
      frqtemp.append(CurFre)
    preFreSets = CurFre
                    
  return frqtemp
    

     


support = 0.005


#将输入数据分区，由于后面要频繁使用。因此缓存起来
#display(dbutils.fs.ls("/FileStore/tables"))
splitedl = sc.textFile('/FileStore/tables/retail.txt',8)
#print(input_rdd.collect())


splitedl = splitedl.map(lambda x:splitlines(x))
splitedl.collect()
transationss = splitedl.map(lambda x: x.split(' '))
transationss.collect()
#print('tran:\n',transationss.collect())
#print('spl:\n',splitedl.collect())

numRecords = float(transationss.count())
freqThre = numRecords * support



######
#print('stage:merge')
"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates = transationss.mapPartitions(lambda x: FindFreInPartition(x,support))
#print('globalcan:\n',global_candidates.collect())  
global_candidates = global_candidates.flatMap(lambda its:[frozenset(it) for it in its]).distinct()
#print('globalcan:\n',global_candidates.collect())    

"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
original_data=sc.broadcast(transationss.collect())
    
"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates.map(lambda x:(x,len([transaction for transaction in original_data.value if x.issubset(frozenset(transaction))])/numRecords))\
    .filter(lambda x: x[1]>=support).collectAsMap()
print ('global_frequent_itemsets:\n',global_frequent_itemsets)

  

   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576167864364-0/PythonShell.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from collections import OrderedDict
global_frequent_itemsets:
 {frozenset({'677'}): 0.012590458474172546, frozenset({'2046'}): 0.007089222113835893, frozenset({'681'}): 0.00798529978902475, frozenset({'53'}): 0.005773462489507952, frozenset({'201'}): 0.012851341847961707, frozenset({'76'}): 0.009856854427077425, frozenset({'251'}): 0.0070438510923073435, frozenset({'766'}): 0.00904017603956353, frozenset({'479'}): 0.010503391483859259, frozenset({'1198'}): 0.007486218552210703, frozenset({'170'}): 0.03515119892924389, frozenset({'178'}): 0.005433179828043829, frozenset({'110'}): 0.03169165853769198, frozenset({'789'}): 0.007350105487625054, frozenset({'379'}): 0.007679045393707039, frozenset({'338'}): 0.014450670356843085, frozenset({'703'}): 0.009142260838002767, frozenset({'589'}): 0.012692543272611783, frozenset({'548'}): 0.012896712869490257, frozenset({'32', '48'}): 0.0911276967400921, frozenset({'32', '225'}): 0.007429504775300016, frozenset({'39', '48'}): 0.33055057734624893, frozenset({'38', '225'}): 0.007724416415235589, frozenset({'39', '225'}): 0.026666817903405096, frozenset({'56', '38'}): 0.005830176266418638, frozenset({'39', '156'}): 0.006601483632403983, frozenset({'38', '48'}): 0.09010684875569973, frozenset({'39', '1146'}): 0.011149928540641093, frozenset({'23', '39'}): 0.005807490755654364, frozenset({'161', '39'}): 0.007055193847689481, frozenset({'39', '237'}): 0.0218801751321431, frozenset({'48', '78'}): 0.009346430434881241, frozenset({'1327', '48'}): 0.01097978720990903, frozenset({'10', '39'}): 0.005126925432726118, frozenset({'264', '48'}): 0.006057031374061387, frozenset({'48', '89'}): 0.031737029559220524, frozenset({'2958', '48'}): 0.008836006442685057, frozenset({'237', '32'}): 0.0064426850570540595, frozenset({'179', '48'}): 0.007077879358453756, frozenset({'740', '39'}): 0.008609151335042309, frozenset({'39', '3270'}): 0.006533427100111159, frozenset({'185', '48'}): 0.009255688391824142, frozenset({'522', '39'}): 0.007350105487625054, frozenset({'271', '41'}): 0.0066241691431682586, frozenset({'19', '48'}): 0.0067035684308432205, frozenset({'101', '48'}): 0.01487035230598217, frozenset({'38', '237'}): 0.007747101925999864, frozenset({'39', '1004'}): 0.006612826387786121, frozenset({'175', '48'}): 0.0055919784033937525, frozenset({'170', '48', '41'}): 0.005580635648011615, frozenset({'310', '39', '48'}): 0.015278691499739117, frozenset({'475', '39', '48'}): 0.012386288877294073, frozenset({'286', '39', '38'}): 0.008257525918196049, frozenset({'592', '39', '48'}): 0.0058415190218007755, frozenset({'39', '2238', '48'}): 0.008938091241124295, frozenset({'39', '258', '48'}): 0.005126925432726118, frozenset({'38', '39', '170', '41'}): 0.006975794560014519, frozenset({'38', '39', '110', '41'}): 0.005796148000272226, frozenset({'374'}): 0.0066241691431682586, frozenset({'12929'}): 0.008416324493545973, frozenset({'39', '806'}): 0.0055919784033937525, frozenset({'39', '49'}): 0.008711236133481546, frozenset({'39', '48', '413'}): 0.008858691953449332, frozenset({'301', '39', '48'}): 0.005365123295751004, frozenset({'286', '39', '38', '48'}): 0.005194981965018943, frozenset({'136'}): 0.005297066763458179, frozenset({'31', '39'}): 0.0061024023955899365, frozenset({'15618'}): 0.0069644518046323815, frozenset({'39', '389'}): 0.006170458927882761, frozenset({'16010', '39', '48'}): 0.0060003175971507, frozenset({'1693'}): 0.005943603820240013, frozenset({'1783'}): 0.006805653229282458, frozenset({'107'}): 0.008677207867335134, frozenset({'161'}): 0.011456182935958802, frozenset({'488'}): 0.00816678387513895, frozenset({'31'}): 0.010435334951566434, frozenset({'740'}): 0.013395794106304303, frozenset({'1146'}): 0.016174769174927973, frozenset({'10'}): 0.008076041832081849, frozenset({'522'}): 0.011047843742201

In [11]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#====================w=5==============
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools
import copy


start = time.clock()
t0 = time.time()



def splitlines(x):
    y = ' '.join(x.split())
    return y
    


def prune(transactions,Ck,support):
    # CSups candidate and supports
    CSups = {}
    for transaction in transactions:
        for candi in Ck:
            candi = frozenset(candi)
            if candi.issubset(frozenset(transaction)):
              if not frozenset(candi) in CSups:
                  CSups[frozenset(candi)] =1
              else:
                  CSups[frozenset(candi)] +=1

                  
    numRecords = float(len(transactions))
    
    # 满足最小支持度的频繁项集和频率
    # FreSups frequent itemset and support
    FreSups = {}
    CurFre = []
    for candi in CSups:
        support_candi = CSups[candi]/numRecords   
        if support_candi >= support:
            CurFre.append(candi)
            FreSups[candi] = support_candi
     
    return CurFre, FreSups
  
def genCk(Lk, k):
    Ck = [] 
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            L1 = list(set(Lk[i]))[: k-2]
            L2 = list(set(Lk[j]))[: k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                Ck.append(frozenset(Lk[i] | Lk[j]))
    return Ck

def FindFreInPartition(iteration,support):
  import itertools
  frqtemp=[]
  preFreSets=[]
  #support = 0.3
  splitedline=list(iteration)
  transactions=copy.deepcopy(splitedline)
  
  numRecords = len(splitedline)
  freqThre = numRecords * support
  #print ("numRecords：" + str(numRecords))
  #print ("frequency threshold：" + str(freqThre))
  #transactions = list(map(lambda x : x.split(' '),splitedline))
                 
         
  #Create candidate_1
  candidate_1 = list(set(list(itertools.chain.from_iterable(transactions))))#distinct
  candidate_1 = list(map(lambda c: [c] ,candidate_1))
  #print(candidate_1)
  CurFre1, FreSups = prune(transactions,candidate_1,support)
  frqtemp.append(frozenset(CurFre1))
  preFreSets = CurFre1
  
         
  #Create CK LK
  K =200 #迭代次数
  for r in range(2,K):
    #print ('round'+str(r)+':\n')
    #print (preFreSets)
    if len(preFreSets)==0:
      break
    candidates = genCk(preFreSets, r)
    CurFre, FreSups = prune(transactions,candidates,support)
    if len(CurFre)!=0:
      frqtemp.append(CurFre)
    preFreSets = CurFre
                    
  return frqtemp
    

     


support = 0.005


#将输入数据分区，由于后面要频繁使用。因此缓存起来
#display(dbutils.fs.ls("/FileStore/tables"))
splitedl = sc.textFile('/FileStore/tables/retail.txt',8)
#print(input_rdd.collect())


splitedl = splitedl.map(lambda x:splitlines(x))
splitedl.collect()
transationss = splitedl.map(lambda x: x.split(' '))
transationss.collect()
#print('tran:\n',transationss.collect())
#print('spl:\n',splitedl.collect())

numRecords = float(transationss.count())
freqThre = numRecords * support



######
#print('stage:merge')
"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates = transationss.mapPartitions(lambda x: FindFreInPartition(x,support))
#print('globalcan:\n',global_candidates.collect())  
global_candidates = global_candidates.flatMap(lambda its:[frozenset(it) for it in its]).distinct()
#print('globalcan:\n',global_candidates.collect())    

"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
original_data=sc.broadcast(transationss.collect())
    
"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates.map(lambda x:(x,len([transaction for transaction in original_data.value if x.issubset(frozenset(transaction))])/numRecords))\
    .filter(lambda x: x[1]>=support).collectAsMap()
print ('global_frequent_itemsets:\n',global_frequent_itemsets)

  

   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576167864364-0/PythonShell.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from collections import OrderedDict
global_frequent_itemsets:
 {frozenset({'677'}): 0.012590458474172546, frozenset({'2046'}): 0.007089222113835893, frozenset({'681'}): 0.00798529978902475, frozenset({'201'}): 0.012851341847961707, frozenset({'76'}): 0.009856854427077425, frozenset({'374'}): 0.0066241691431682586, frozenset({'251'}): 0.0070438510923073435, frozenset({'766'}): 0.00904017603956353, frozenset({'479'}): 0.010503391483859259, frozenset({'1198'}): 0.007486218552210703, frozenset({'170'}): 0.03515119892924389, frozenset({'110'}): 0.03169165853769198, frozenset({'789'}): 0.007350105487625054, frozenset({'338'}): 0.014450670356843085, frozenset({'703'}): 0.009142260838002767, frozenset({'589'}): 0.012692543272611783, frozenset({'548'}): 0.012896712869490257, frozenset({'38', '48'}): 0.09010684875569973, frozenset({'39', '48'}): 0.33055057734624893, frozenset({'101', '48'}): 0.01487035230598217, frozenset({'48', '89'}): 0.031737029559220524, frozenset({'32', '48'}): 0.0911276967400921, frozenset({'179', '48'}): 0.007077879358453756, frozenset({'38', '237'}): 0.007747101925999864, frozenset({'39', '237'}): 0.0218801751321431, frozenset({'38', '225'}): 0.007724416415235589, frozenset({'39', '225'}): 0.026666817903405096, frozenset({'161', '39'}): 0.007055193847689481, frozenset({'237', '32'}): 0.0064426850570540595, frozenset({'23', '39'}): 0.005807490755654364, frozenset({'48', '19'}): 0.0067035684308432205, frozenset({'740', '39'}): 0.008609151335042309, frozenset({'522', '39'}): 0.007350105487625054, frozenset({'39', '1146'}): 0.011149928540641093, frozenset({'271', '41'}): 0.0066241691431682586, frozenset({'1327', '48'}): 0.01097978720990903, frozenset({'39', '1004'}): 0.006612826387786121, frozenset({'185', '48'}): 0.009255688391824142, frozenset({'225', '32'}): 0.007429504775300016, frozenset({'39', '3270'}): 0.006533427100111159, frozenset({'2958', '48'}): 0.008836006442685057, frozenset({'286', '39', '38'}): 0.008257525918196049, frozenset({'475', '39', '48'}): 0.012386288877294073, frozenset({'170', '48', '41'}): 0.005580635648011615, frozenset({'592', '39', '48'}): 0.0058415190218007755, frozenset({'310', '39', '48'}): 0.015278691499739117, frozenset({'301', '39', '48'}): 0.005365123295751004, frozenset({'39', '2238', '48'}): 0.008938091241124295, frozenset({'38', '39', '110', '41'}): 0.005796148000272226, frozenset({'38', '39', '170', '41'}): 0.006975794560014519, frozenset({'12929'}): 0.008416324493545973, frozenset({'178'}): 0.005433179828043829, frozenset({'379'}): 0.007679045393707039, frozenset({'264', '48'}): 0.006057031374061387, frozenset({'48', '78'}): 0.009346430434881241, frozenset({'39', '806'}): 0.0055919784033937525, frozenset({'39', '49'}): 0.008711236133481546, frozenset({'56', '38'}): 0.005830176266418638, frozenset({'39', '156'}): 0.006601483632403983, frozenset({'10', '39'}): 0.005126925432726118, frozenset({'175', '48'}): 0.0055919784033937525, frozenset({'39', '48', '413'}): 0.008858691953449332, frozenset({'286', '39', '38', '48'}): 0.005194981965018943, frozenset({'136'}): 0.005297066763458179, frozenset({'31', '39'}): 0.0061024023955899365, frozenset({'39', '258', '48'}): 0.005126925432726118, frozenset({'53'}): 0.005773462489507952, frozenset({'15618'}): 0.0069644518046323815, frozenset({'16010', '39', '48'}): 0.0060003175971507, frozenset({'39', '389'}): 0.006170458927882761, frozenset({'1783'}): 0.006805653229282458, frozenset({'107'}): 0.008677207867335134, frozenset({'161'}): 0.011456182935958802, frozenset({'1020'}): 0.005274381252693905, frozenset({'488'}): 0.00816678387513895, frozenset({'31'}): 0.010435334951566434, frozenset({'94'}): 0.0058528617771829135, frozenset({'740'}): 0.013395794106304303, frozenset({'1146'}): 0.016174769174927973, frozenset({'522'}): 0.01104784374220

In [12]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#====================w=2==============
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time
import itertools
import copy


start = time.clock()
t0 = time.time()



def splitlines(x):
    y = ' '.join(x.split())
    return y
    


def prune(transactions,Ck,support):
    # CSups candidate and supports
    CSups = {}
    for transaction in transactions:
        for candi in Ck:
            candi = frozenset(candi)
            if candi.issubset(frozenset(transaction)):
              if not frozenset(candi) in CSups:
                  CSups[frozenset(candi)] =1
              else:
                  CSups[frozenset(candi)] +=1

                  
    numRecords = float(len(transactions))
    
    # 满足最小支持度的频繁项集和频率
    # FreSups frequent itemset and support
    FreSups = {}
    CurFre = []
    for candi in CSups:
        support_candi = CSups[candi]/numRecords   
        if support_candi >= support:
            CurFre.append(candi)
            FreSups[candi] = support_candi
     
    return CurFre, FreSups
  
def genCk(Lk, k):
    Ck = [] 
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            L1 = list(set(Lk[i]))[: k-2]
            L2 = list(set(Lk[j]))[: k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                Ck.append(frozenset(Lk[i] | Lk[j]))
    return Ck

def FindFreInPartition(iteration,support):
  import itertools
  frqtemp=[]
  preFreSets=[]
  #support = 0.3
  splitedline=list(iteration)
  transactions=copy.deepcopy(splitedline)
  
  numRecords = len(splitedline)
  freqThre = numRecords * support
  #print ("numRecords：" + str(numRecords))
  #print ("frequency threshold：" + str(freqThre))
  #transactions = list(map(lambda x : x.split(' '),splitedline))
                 
         
  #Create candidate_1
  candidate_1 = list(set(list(itertools.chain.from_iterable(transactions))))#distinct
  candidate_1 = list(map(lambda c: [c] ,candidate_1))
  #print(candidate_1)
  CurFre1, FreSups = prune(transactions,candidate_1,support)
  frqtemp.append(frozenset(CurFre1))
  preFreSets = CurFre1
  
         
  #Create CK LK
  K =200 #迭代次数
  for r in range(2,K):
    #print ('round'+str(r)+':\n')
    #print (preFreSets)
    if len(preFreSets)==0:
      break
    candidates = genCk(preFreSets, r)
    CurFre, FreSups = prune(transactions,candidates,support)
    if len(CurFre)!=0:
      frqtemp.append(CurFre)
    preFreSets = CurFre
                    
  return frqtemp
    

     


support = 0.005


#将输入数据分区，由于后面要频繁使用。因此缓存起来
#display(dbutils.fs.ls("/FileStore/tables"))
splitedl = sc.textFile('/FileStore/tables/retail.txt',8)
#print(input_rdd.collect())


splitedl = splitedl.map(lambda x:splitlines(x))
splitedl.collect()
transationss = splitedl.map(lambda x: x.split(' '))
transationss.collect()
#print('tran:\n',transationss.collect())
#print('spl:\n',splitedl.collect())

numRecords = float(transationss.count())
freqThre = numRecords * support



######
#print('stage:merge')
"""For each partition of transactions, get the local frequent itemset.
       Merge the local frequent itemsets as the global candidates."""
global_candidates = transationss.mapPartitions(lambda x: FindFreInPartition(x,support))
#print('globalcan:\n',global_candidates.collect())  
global_candidates = global_candidates.flatMap(lambda its:[frozenset(it) for it in its]).distinct()
#print('globalcan:\n',global_candidates.collect())    

"""Broadcast transactions to each worker node. Then every worker can scan transactions
       directly from the broadcast value."""
original_data=sc.broadcast(transationss.collect())
    
"""For each candidate find its frequency by scanning the transactions.
    Filter candidates by calculating their support.Finally collect the rdd
    as map where the key is the frequent itemset and the value is the support
    of the frequent itemset."""
global_frequent_itemsets=global_candidates.map(lambda x:(x,len([transaction for transaction in original_data.value if x.issubset(frozenset(transaction))])/numRecords))\
    .filter(lambda x: x[1]>=support).collectAsMap()
print ('global_frequent_itemsets:\n',global_frequent_itemsets)

  

   

    
end = time.clock()
print (end - start, "seconds process time")
print (time.time() - t0, "seconds wall time")




/local_disk0/tmp/1576167864364-0/PythonShell.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 from collections import OrderedDict
global_frequent_itemsets:
 {frozenset({'677'}): 0.012590458474172546, frozenset({'2046'}): 0.007089222113835893, frozenset({'681'}): 0.00798529978902475, frozenset({'53'}): 0.005773462489507952, frozenset({'201'}): 0.012851341847961707, frozenset({'76'}): 0.009856854427077425, frozenset({'251'}): 0.0070438510923073435, frozenset({'766'}): 0.00904017603956353, frozenset({'479'}): 0.010503391483859259, frozenset({'1198'}): 0.007486218552210703, frozenset({'170'}): 0.03515119892924389, frozenset({'178'}): 0.005433179828043829, frozenset({'110'}): 0.03169165853769198, frozenset({'789'}): 0.007350105487625054, frozenset({'379'}): 0.007679045393707039, frozenset({'338'}): 0.014450670356843085, frozenset({'703'}): 0.009142260838002767, frozenset({'589'}): 0.012692543272611783, frozenset({'548'}): 0.012896712869490257, frozenset({'32', '48'}): 0.0911276967400921, frozenset({'32', '225'}): 0.007429504775300016, frozenset({'39', '48'}): 0.33055057734624893, frozenset({'38', '225'}): 0.007724416415235589, frozenset({'39', '225'}): 0.026666817903405096, frozenset({'56', '38'}): 0.005830176266418638, frozenset({'39', '156'}): 0.006601483632403983, frozenset({'38', '48'}): 0.09010684875569973, frozenset({'39', '1146'}): 0.011149928540641093, frozenset({'23', '39'}): 0.005807490755654364, frozenset({'161', '39'}): 0.007055193847689481, frozenset({'39', '237'}): 0.0218801751321431, frozenset({'48', '78'}): 0.009346430434881241, frozenset({'1327', '48'}): 0.01097978720990903, frozenset({'10', '39'}): 0.005126925432726118, frozenset({'264', '48'}): 0.006057031374061387, frozenset({'48', '89'}): 0.031737029559220524, frozenset({'2958', '48'}): 0.008836006442685057, frozenset({'237', '32'}): 0.0064426850570540595, frozenset({'179', '48'}): 0.007077879358453756, frozenset({'740', '39'}): 0.008609151335042309, frozenset({'39', '3270'}): 0.006533427100111159, frozenset({'185', '48'}): 0.009255688391824142, frozenset({'522', '39'}): 0.007350105487625054, frozenset({'271', '41'}): 0.0066241691431682586, frozenset({'19', '48'}): 0.0067035684308432205, frozenset({'101', '48'}): 0.01487035230598217, frozenset({'38', '237'}): 0.007747101925999864, frozenset({'39', '1004'}): 0.006612826387786121, frozenset({'175', '48'}): 0.0055919784033937525, frozenset({'170', '48', '41'}): 0.005580635648011615, frozenset({'310', '39', '48'}): 0.015278691499739117, frozenset({'475', '39', '48'}): 0.012386288877294073, frozenset({'286', '39', '38'}): 0.008257525918196049, frozenset({'592', '39', '48'}): 0.0058415190218007755, frozenset({'39', '2238', '48'}): 0.008938091241124295, frozenset({'39', '258', '48'}): 0.005126925432726118, frozenset({'38', '39', '170', '41'}): 0.006975794560014519, frozenset({'38', '39', '110', '41'}): 0.005796148000272226, frozenset({'136'}): 0.005297066763458179, frozenset({'374'}): 0.0066241691431682586, frozenset({'39', '389'}): 0.006170458927882761, frozenset({'39', '49'}): 0.008711236133481546, frozenset({'31', '39'}): 0.0061024023955899365, frozenset({'39', '48', '413'}): 0.008858691953449332, frozenset({'301', '39', '48'}): 0.005365123295751004, frozenset({'286', '38', '39', '48'}): 0.005194981965018943, frozenset({'12929'}): 0.008416324493545973, frozenset({'39', '806'}): 0.0055919784033937525, frozenset({'15618'}): 0.0069644518046323815, frozenset({'16010', '39', '48'}): 0.0060003175971507, frozenset({'1783'}): 0.006805653229282458, frozenset({'107'}): 0.008677207867335134, frozenset({'161'}): 0.011456182935958802, frozenset({'1020'}): 0.005274381252693905, frozenset({'488'}): 0.00816678387513895, frozenset({'31'}): 0.010435334951566434, frozenset({'94'}): 0.0058528617771829135, frozenset({'740'}): 0.013395794106304303, frozenset({'1146'}): 0.016174769174927973, frozenset({'522'}): 0.01104784374220